<a href="https://colab.research.google.com/github/JevinKOpenEyes/colab-copies/blob/main/working%20summary%20and%20score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q auto-gptq transformers tika aspose-words accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 61.0 MB/s eta 0:00:00
ERR

In [ ]:
import requests
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
# This line gets ssf6.pdf files metadata from the google drive URI
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/66acaf7a1571ea56e405e9c2")
resp = request.json()

# Getting file metadata
file_path = (resp["data"]["file_path"])
file_name = (resp["data"]["file_name"])
_id = (resp["data"]["_id"])
file_type = (resp["data"]["file_type"])
user_file_name = (resp["data"]["file_name"])
type_of_question = (resp["data"]["type_of_question"])

In [ ]:
if file_type == "pdf" or file_type == "txt":
    Text_pdf = []
    # Read PDF file
    data = parser.from_file(file_path, xmlContent=True)
    xhtml_data = BeautifulSoup(data['content'])
    for i, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
        # Parse PDF data using TIKA (xml/html)
        # It's faster and safer to create a new buffer than truncating it
        # https://stackoverflow.com/questions/4330812/how-do-i-clear-a-stringio-object
        _buffer = StringIO()
        _buffer.write(str(content))
        parsed_content = parser.from_buffer(_buffer.getvalue())

        # Add pages
        if parsed_content['content']!=None:
            text = parsed_content['content'].strip()
            Text_pdf.append(text)

2024-08-05 05:23:40,791 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.
INFO:tika.tika:Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.
2024-08-05 05:23:43,066 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2024-08-05 05:23:43,589 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-se

In [ ]:
from typing import List, Dict
from tqdm import tqdm
from transformers import pipeline

candidate_labels = ["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"]

def get_pdf_by_id(hashcode: str) -> List[str]:
    request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/%s" % hashcode)
    resp = request.json()

    # Getting file metadata
    file_path = (resp["data"]["file_path"])
    file_name = (resp["data"]["file_name"])
    _id = (resp["data"]["_id"])
    file_type = (resp["data"]["file_type"])
    user_file_name = (resp["data"]["file_name"])
    type_of_question = (resp["data"]["type_of_question"])

    if file_type == "pdf" or file_type == "txt":
        Text_pdf = []
        # Read PDF file
        data = parser.from_file(file_path, xmlContent=True)
        xhtml_data = BeautifulSoup(data['content'])
        for i, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
            # Parse PDF data using TIKA (xml/html)
            # It's faster and safer to create a new buffer than truncating it
            # https://stackoverflow.com/questions/4330812/how-do-i-clear-a-stringio-object
            _buffer = StringIO()
            _buffer.write(str(content))
            parsed_content = parser.from_buffer(_buffer.getvalue())

            # Add pages
            if parsed_content['content']!=None:
                text = parsed_content['content'].strip()
                Text_pdf.append(text)

        return Text_pdf

def summarize_and_tag(texts: List[str], summary_model: str,
                      zs_model: str, pdf_file_name: str, candidate_labels=candidate_labels) -> List[Dict[str, str]]:

    summarizer = pipeline("summarization", model=summary_model, do_sample=True)
    zs_er = pipeline("zero-shot-classification", model=zs_model)

    summary = summarizer(text, max_length=300, min_length=100, do_sample=False)
    summary = " ".join(item["summary_text"] for item in summary)

    classification = zs_er(summary, candidate_labels=candidate_labels)
    return {"summary": summary, "classification_result": classification, "file_name": pdf_file_name}


In [ ]:
hashcodes = {
    "S9549": "66b07cb4c3a27f936a05d5b2",
    ""
}

summary_model = "facebook/bart-large-cnn"
zs_model = "tasksource/deberta-base-long-nli"

outputs = []
for pdf_file, hashcode in hashcodes.items():
    texts = get_pdf_by_id(hashcode)
    outputs.append(summarize_and_tag(texts, summary_model=summary_model, zs_model=zs_model, pdf_file_name=pdf_file))


2024-08-05 07:35:01,993 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/u/0/uc?id=19ZV_f_CQYUHqwmrT6V7a56NEJEejB0Gn&export=download to /tmp/u-0-uc.
INFO:tika.tika:Retrieving https://drive.google.com/u/0/uc?id=19ZV_f_CQYUHqwmrT6V7a56NEJEejB0Gn&export=download to /tmp/u-0-uc.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
import pprint
for output in outputs:
    pprint.pprint(output["file_name"])
    pprint.pprint(output["summary"])
    pprint.pprint(output["classification_result"])

('This application may be harmful to your mental health. A social media '
 'platform that complies with subsections 84, 85, and (3) may post a statement '
 'confirming such compliance on 85 its Internet homepage or platform user '
 'login page. A public K-12 school may not: (a)  Use or have an account on a '
 'social media. platform that 88, 89, or 90 does not have a statement pursuant '
 'to subsection (4) on its 89, 90, or 91, 91, 92, 93, 94, 95, 96, 97, 98, 99.')
{'labels': ['Social',
            'Education',
            'Healthcare',
            'Privacy',
            'AG',
            'Other',
            'Government',
            'Telecom',
            'Data',
            'State Response',
            'Labor',
            'Energy',
            'Budget',
            'Military',
            'Transportation'],
 'scores': [0.29423344135284424,
            0.23428034782409668,
            0.12644623219966888,
            0.0993882566690445,
            0.06643413007259369,
           